In [11]:
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime
import numpy as np

# -----------------------------
# 1. Station coordinates
# -----------------------------
stations = {
    "VKCL": {"lat": 50.956, "lon": 7.014},
    "VKTU": {"lat": 50.936, "lon": 6.961},
    "CHOR": {"lat": 50.99987, "lon": 6.88350},
    "RODE": {"lat": 50.87044, "lon": 6.96730},
}

# -----------------------------
# 2. PM2.5 → AQI conversion
# -----------------------------
def pm25_to_aqi(pm):
    breakpoints = [
        (0.0, 12.0, 0, 50),
        (12.1, 35.4, 51, 100),
        (35.5, 55.4, 101, 150),
        (55.5, 150.4, 151, 200),
        (150.5, 250.4, 201, 300),
        (250.5, 350.4, 301, 400),
        (350.5, 500.4, 401, 500),
    ]
    for c_low, c_high, a_low, a_high in breakpoints:
        if c_low <= pm <= c_high:
            return ((a_high - a_low) / (c_high - c_low)) * (pm - c_low) + a_low
    return np.nan

# -----------------------------
# 3. Load CSVs
# -----------------------------
files = {
    "VKCL": "VKCL.csv",
    "VKTU": "VKTU.csv",
    "CHOR": "CHOR.csv",
    "RODE": "RODE.csv"
}

frames = []

for name, path in files.items():
    df = pd.read_csv(path)
    df.columns = ["datum_beginn", "datum_ende", "pm25"]
    df["datum_beginn"] = pd.to_datetime(df["datum_beginn"], errors="coerce")
    df["pm25"] = pd.to_numeric(df["pm25"], errors="coerce")
    df["aqi"] = df["pm25"].apply(pm25_to_aqi)
    df["station"] = name
    df["lat"] = stations[name]["lat"]
    df["lon"] = stations[name]["lon"]
    frames.append(df)

data = pd.concat(frames)
data = data.dropna(subset=["datum_beginn", "lat", "lon"])
data = data.sort_values("datum_beginn")
data["aqi"] = pd.to_numeric(data["aqi"], errors="coerce").fillna(0.1)

# -----------------------------
# 4. Create time slices
# -----------------------------
timestamps = sorted(data["datum_beginn"].unique())
time_slices = []

for t in timestamps:
    df_t = data[data["datum_beginn"] == t]
    heat_data = df_t[["lat", "lon", "aqi"]].values.tolist()
    time_slices.append(heat_data)

# -----------------------------
# 5. Build base map
# -----------------------------
m = folium.Map(location=[50.94, 6.96], zoom_start=11)

# Add green location markers with AQI popups and permanent labels
latest_data = data.groupby("station").last().reset_index()
for _, row in latest_data.iterrows():
    # Green location marker
    folium.Marker(
        location=[row["lat"], row["lon"]],
        icon=folium.Icon(color="green", icon="glyphicon-map-marker"),
        popup=folium.Popup(f"<b>{row['station']}</b><br>AQI: {row['aqi']:.1f}", max_width=200)
    ).add_to(m)
    
    # Permanent station name label
    folium.map.Marker(
        [row["lat"], row["lon"]],
        icon=folium.DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,-10),
            html=f'<div style="font-size:12px; color:black; font-weight:bold">{row["station"]}</div>'
        )
    ).add_to(m)

# -----------------------------
# 6. HeatMap with time slider
# -----------------------------
HeatMapWithTime(
    time_slices,
    index=[str(t) for t in timestamps],
    radius=60,
    min_opacity=0.5,
    max_opacity=0.95,
    use_local_extrema=True,
    gradient={
        0.0: "green",
        0.3: "yellow",
        0.6: "orange",
        1.0: "red"
    }
).add_to(m)

# -----------------------------
# 7. Legend (top-right)
# -----------------------------
legend_html = """
<div style="
    position: fixed;
    top: 50px;
    right: 50px;
    width: 160px;
    height: 160px;
    z-index:9999;
    font-size:14px;
    background-color:white;
    border:2px solid grey;
    padding: 10px;
    ">
<b>AQI Legend</b><br>
<span style="background-color:green;color:white;padding:3px;">0-50 Good</span><br>
<span style="background-color:yellowgreen;color:white;padding:3px;">51-100 Moderate</span><br>
<span style="background-color:yellow;color:white;padding:3px;">101-150 Unhealthy SG</span><br>
<span style="background-color:orange;color:white;padding:3px;">151-200 Unhealthy</span><br>
<span style="background-color:red;color:white;padding:3px;">201+ Very Unhealthy</span>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# -----------------------------
# 8. Save map
# -----------------------------
m.save("Cologne_AQI_heatmap.html")
print("✔ Map created: cologne_aqi_heatmap_final.html")


✔ Map created: cologne_aqi_heatmap_final.html
